In [37]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
import numpy as np

In [43]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=40,
    height_shift_range=0.2,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_path = 'dataset/'

train_dataset = train_datagen.flow_from_directory(train_path, 
                                               target_size=IMG_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training') 

test_dataset = train_datagen.flow_from_directory(train_path, 
                                           target_size=IMG_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

Found 70306 images belonging to 38 classes.
Found 17561 images belonging to 38 classes.


In [46]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x) 
x = Dropout(0.2)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_dataset.num_classes, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_dataset,
    epochs=3,
    verbose=1
)

Epoch 1/3
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 327s 148ms/step - accuracy: 0.6865 - loss: 1.0622
Epoch 2/3
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 322s 146ms/step - accuracy: 0.8125 - loss: 0.5857
Epoch 3/3
2198/2198 ━━━━━━━━━━━━━━━━━━━━ 329s 149ms/step - accuracy: 0.8369 - loss: 0.5141


In [47]:
model.save('model.h5')

In [48]:
model = load_model('model.h5')

loss, accuracy = model.evaluate(test_dataset)

print(f'Test accuracy: {accuracy:.4f}')

549/549 ━━━━━━━━━━━━━━━━━━━━ 81s 147ms/step - accuracy: 0.8848 - loss: 0.3610
Test accuracy: 0.8767


In [49]:
import numpy as np


predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)


549/549 ━━━━━━━━━━━━━━━━━━━━ 78s 142ms/step


In [52]:
from sklearn.metrics import accuracy_score

true_labels = test_dataset.classes
accuracy = accuracy_score(true_labels, predicted_classes)
print('Accuracy:', accuracy)

Accuracy: 0.8743237856614088


In [55]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       504
           1       0.92      0.81      0.86       496
           2       0.92      0.95      0.94       440
           3       0.86      0.89      0.88       502
           4       0.88      0.84      0.86       454
           5       0.96      0.94      0.95       420
           6       0.96      0.84      0.90       456
           7       0.86      0.83      0.85       410
           8       0.99      0.97      0.98       476
           9       0.86      0.92      0.89       477
          10       0.99      0.98      0.99       464
          11       0.82      0.90      0.86       472
          12       0.93      0.84      0.88       480
          13       0.99      0.94      0.96       430
          14       0.92      0.95      0.94       423
          15       0.97      0.98      0.98       502
          16       0.93      0.88      0.90       459
          17       0.85    